## Identifying handwritten digits using Logistic Regression in PyTorch

### Import Libraries

In [31]:
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable

In [32]:
device = 'cuda' if torch.cuda.is_available() else 'gpu'
device

'cuda'

### Reading the Data

In [33]:
# Hyper Parameters  
input_size = 784 # Image size is 28x28
num_classes = 10 # 10 digits
num_epochs = 5
batch_size = 100 # prevent memory overflow
learning_rate = 0.001
path = r'../dataset'

In [34]:
# MNIST Dataset (Images and Labels) 
train_dataset = dsets.MNIST(root = path,  
                            train = True,  
                            transform = transforms.ToTensor(), 
                            download = True) 
  
test_dataset = dsets.MNIST(root = path,  
                           train = False,  
                           transform = transforms.ToTensor()) 
  
# Dataset Loader (Input Pipeline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,  
                                           batch_size = batch_size,  
                                           shuffle = True) 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,  
                                          batch_size = batch_size,  
                                          shuffle = False)

### Build Model

In [35]:
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super().__init__() 
        self.linear = nn.Linear(input_size, num_classes)
  
    def forward(self, x): 
        out = self.linear(x) # the softmax is internally calculated during each forward pass and hence we do not need to specify it inside the forward() function
        return out 

In [36]:
model = LogisticRegression(input_size, num_classes).to(device)

### Loss Function and Optimizer

In [37]:
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) 

### Training

1. Reset all gradients to 0.
2. Make a forward pass.
3. Calculate the loss.
4. Perform backpropagation.
5. Update all weights.

In [38]:
# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.view(-1, 28*28)  # Flatten the images
        
        # Move images and labels to the appropriate device (e.g., GPU if available)
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass: compute the model output
        outputs = model(images)
        
        # Compute the loss
        loss = loss_fn(outputs, labels)
        
        # Zero the gradients before running the backward pass
        optimizer.zero_grad()
        
        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        
        # Perform a single optimization step (parameter update)
        optimizer.step()
        
        # Print statistics
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch + 1, num_epochs, i + 1, len(train_dataset) // batch_size, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.1836
Epoch [1/5], Step [200/600], Loss: 2.0581
Epoch [1/5], Step [300/600], Loss: 1.9743
Epoch [1/5], Step [400/600], Loss: 1.9440
Epoch [1/5], Step [500/600], Loss: 1.8317
Epoch [1/5], Step [600/600], Loss: 1.7531
Epoch [2/5], Step [100/600], Loss: 1.6835
Epoch [2/5], Step [200/600], Loss: 1.6919
Epoch [2/5], Step [300/600], Loss: 1.5906
Epoch [2/5], Step [400/600], Loss: 1.5961
Epoch [2/5], Step [500/600], Loss: 1.5112
Epoch [2/5], Step [600/600], Loss: 1.4292
Epoch [3/5], Step [100/600], Loss: 1.3742
Epoch [3/5], Step [200/600], Loss: 1.3390
Epoch [3/5], Step [300/600], Loss: 1.4428
Epoch [3/5], Step [400/600], Loss: 1.3021
Epoch [3/5], Step [500/600], Loss: 1.1554
Epoch [3/5], Step [600/600], Loss: 1.3362
Epoch [4/5], Step [100/600], Loss: 1.1711
Epoch [4/5], Step [200/600], Loss: 1.1827
Epoch [4/5], Step [300/600], Loss: 1.0618
Epoch [4/5], Step [400/600], Loss: 1.1382
Epoch [4/5], Step [500/600], Loss: 1.1993
Epoch [4/5], Step [600/600], Loss:

### Testing the model

In [28]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    
    # Move images and labels to the appropriate device (e.g., GPU if available)
    images = images.to(device)
    labels = labels.to(device)
    
    # Reshape and predict
    images = images.view(-1, 28 * 28)
    outputs = model(images)
    
    # Get predictions from the maximum value
    _, predicted = torch.max(outputs, 1)
    
    # Total number of labels
    total += labels.size(0)
    
    # Total correct predictions
    correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print('Accuracy of the model on the 10000 test images: %d %%' % accuracy)

Accuracy of the model on the 10000 test images: 82 %
